In [1]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [14]:
# !pip install langchain
# !pip install langchain_google_genai
# !pip install google-generativeai
# !pip install faiss-cpu
# !pip install chromadb

In [2]:
#google key configure
api_file='api_key.txt'
with open(api_file,'r') as file:
    api_key=file.read()
    
    
os.environ['GOOGLE_API_KEY'] = api_key

In [3]:
#intilise the pypdfraeder and reading the documents
pdfs=['attention_you_need.pdf','Economic-report U.S. Electronics Manufacturing Sector.pdf']
text=""

#for loop for the pages
for pdf in pdfs:
    pdf_reader= PdfReader(pdf)
    for page in pdf_reader.pages:
        #aappending the text
        text+= page.extract_text()

In [4]:
text

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and con

In [5]:
#function for the chunks 
def get_chunks(text):
    #text splitter class initialize 
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000,chunk_overlap=1000)
    #splitting the text for the chunk size of 1000
    chunks=text_splitter.split_text(text)
    return chunks

In [6]:
#initialize the get_chunks function for getting chunks
chunks=get_chunks(text)

In [7]:
chunks

['Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and co

In [8]:
#vector_store by using the FAISS
def get_vector_store(text_chunks):
    #embedding model for the creating embedding
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          api_key=os.environ['GOOGLE_API_KEY'] )
    #vector_store
    vector_store=FAISS.from_texts(text_chunks,embedding=embedding)
    return vector_store

In [9]:
#initialize the get_vector_store for vectore_store
vectore_store=get_vector_store(chunks)

In your prompt_template, the {context} placeholder is where the context will be inserted when the chain object processes a question. When you call chain({"input_documents": docs, "question": user_question}, return_only_outputs=True), the chain object uses the prompt_template to format the context and question, and then generates the answer based on this formatted input.

So, in your case, the context comes from the vectore_store.similarity_search(user_question) call, where you find the most similar context based on the user's question. This context is then used in the prompt_template to generate the final input for the question-answering model.

In [10]:

def question_answer(user_question):
    #defing the prompt for understanding the context and how to use it 
    prompt_template='''Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
                        provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n\n\n
                        Context:\n{context}\n
                        Question:\n{question}\n

                         Answer:

                         '''
    #initialize the gemini pro model for the generating the text
    #The temperature parameter controls the randomness of the generated text. 
    #The top_k parameter specifies the number of top words to consider for the next word generation. 
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3,top_k=5)

    # specifying how the context and question should be formatted. 
    #The PromptTemplate class is used to define a template for generating prompts
    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

    #load_qa_chain is responsible for loading a question-answering (QA) chain using the load_qa_chain function. 
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    #getting context from the vector_store by similarity search by user question
    context = vectore_store.similarity_search(user_question)

    #genearting the response from the model
    response = chain({"input_documents": context, "question": user_question}, return_only_outputs=True,)

    # Display response
    return print("Reply:\n\n", response["output_text"])


In [12]:
#use question for generating the 
user_question=input("Enter your question:\n")
#calling the fnction for generating text
question_answer(user_question)

Enter your question:
Electronics Investment by End-Use Sectors?
Reply:

 Equipment investment helps end-use industries maintain or improve their capabilities. In many industries, 
electronics play a vital role in ensuring that those firms remain competitive. This section provides a detailed 
examination of these equipment expenditures by three major consumers of manufactured electronics in 2017: the 
automotive, aerospace, and medical services sectors.


# CHROMADB VECTORE STORE


In [51]:
from langchain_community.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter,TextSplitter,CharacterTextSplitter

In [61]:
from langchain.document_loaders import PyPDFDirectoryLoader
# loader = PyPDFDirectoryLoader(pdf_folder_path)

In [65]:
def load_folder(folder):
    loader = PyPDFDirectoryLoader(folder)
    documents = loader.load()
    return documents

documents = load_folder(r"C:\Users\vishnu.singh\Downloads\NLP PRACTISE\pdf")

len(documents)

69

In [66]:
documents

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing 

In [67]:
def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    docs=text_splitter.split_documents(documents)
    return docs

In [68]:
docs=split_documents(documents)
docs

[Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing 

In [69]:

#vector_store by using the FAISS
def get_vector_store(text_chunks):
    #embedding model for the creating embedding
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          api_key=os.environ['GOOGLE_API_KEY'] )
    #vector_store
    db=Chroma.from_documents(text_chunks,embedding=embedding)
    return db


In [70]:
db=get_vector_store(docs)


In [73]:
def question_answer_db(user_question):
    #defing the prompt for understanding the context and how to use it 
    prompt_template='''Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
                        provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n\n\n
                        Context:\n{context}\n
                        Question:\n{question}\n

                         Answer:

                         '''
    #initialize the gemini pro model for the generating the text
    #The temperature parameter controls the randomness of the generated text. 
    #The top_k parameter specifies the number of top words to consider for the next word generation. 
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3,top_k=5)

    # specifying how the context and question should be formatted. 
    #The PromptTemplate class is used to define a template for generating prompts
    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

    #load_qa_chain is responsible for loading a question-answering (QA) chain using the load_qa_chain function. 
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    #getting context from the vector_store by similarity search by user question
    context = db.similarity_search(user_question)

    #genearting the response from the model
    response = chain({"input_documents": context, "question": user_question}, return_only_outputs=True,)

    # Display response
    return print("Reply:\n\n", response["output_text"])


In [77]:
#use question for generating the 
user_question=input("Enter your question:\n")
#calling the fnction for generating text
question_answer_db(user_question)

Enter your question:
Explain the model articture of transformers in detiled way?
Reply:

 The Transformer model architecture consists of an encoder and a decoder, each of which is composed of a stack of identical layers. Each layer in the encoder has two sub-layers: a multi-head self-attention mechanism and a simple, position-wise fully connected feed-forward network. Each layer in the decoder also has two sub-layers: a masked multi-head self-attention mechanism and a multi-head encoder-decoder attention mechanism, followed by a simple, position-wise fully connected feed-forward network.

The encoder maps an input sequence of symbol representations (x1, ..., x n) to a sequence of continuous representations z= (z1, ..., z n). Given z, the decoder then generates an output sequence (y1, ..., y m) of symbols one element at a time. At each step, the model is auto-regressive [10], consuming the previously generated symbols as additional input when generating the next.

The multi-head self-at